<a href="https://colab.research.google.com/github/Fiarma5233/Data-Science/blob/master/ETL_Complet_Pratice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
'''ETL
Extraction des donnees d'un site web
Transformation des prix en les multipliant par un taux donne
Chargement des donnees au format csv,excel et dans une db sql3lite

Ecriture des requetes pour interroger la db
Journalisation des acctions'''

"ETL\nExtraction des donnees d'un site web\nTransformation des prix en les multipliant par un taux donne\nChargement des donnees au format csv,excel et dans une db sql3lite\n\nEcriture des requetes pour interroger la db\nJournalisation des acctions"

In [3]:
# importation de bibliotheques

import requests
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime
from bs4 import BeautifulSoup

# data urls
url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"

# Echange rate csv path
url3 = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv'

# table attributs
table_attrib = ['Name', 'MC_USD_Billion'] # attributs qu'on doit extraitre de la page web

# les attributs qu'on doit avoir apres la transformation(conversion en d'autres devises)
table_convert_attrib = ['Name', 'MC_USD_Billion', 'MC_GBP_Billion', 'MC_EUR_Billion', 'MC_INR_Billion']

#  chemin du fichier csv en sortie
csv_path ='./Largest_banks_data.csv'

# nom de la base de donnnees
db_name ='Banks.db'

# nom de la  table
table_name = 'Largest_banks'

# connexion a la base de donnees sqlite3
sql_connection = sqlite3.connect(db_name)

# fichier de journalisation
log_file = 'code_log.txt'


# fonction d'extraction des donnees
def extract(url,  table_attrib):
    # on recupere la page en text grace a la requete envoyee a cette url
    page = requests.get(url).text

    # on cree un objet beautifulsoup
    data = BeautifulSoup(page, 'html.parser')

    # on definit un df vide
    df = pd.DataFrame(columns=table_attrib)

    # on recupere notre tableau
    tables = data.find_all('tbody')

    # on recupere toutes les lignes de notre tableau
    rows = tables[0].find_all('tr')

    # boucle for
    for row in rows:
        # on recupere toutes les cellules de chaque ligne
        col = row.find_all('td')

        # on verifie qu'il existe des cellules
        if len(col) !=0 :
            # on cree un dictionnaire(cle, valeur) pour recuperer chaque valeur dans la cellule correspondant a la colonne
            dict_values = {
                "Name" : col[1].text.strip() ,
                "MC_USD_Billion" : col[2].text.strip()
            }


            # creation d'un df avec les valeurs recuperees
            df1 = pd.DataFrame(dict_values, index=[0])

            # concatenation des 2 df
            df = pd.concat([df, df1], ignore_index=True)

    return df

extracted_data = extract(url, table_attrib)
print(extracted_data)

# fonction de journalisation
def log_progress(message):
    # format du horodatage
    timestamp_format = '%Y-%h-%d-%H:%M:%S'

    # horodatage actuel
    now = datetime.now()

    # conversion du format de l'horodatage en chaine de caracteres
    timestamp = now.strftime(timestamp_format)

    # ouverture puis ecriture dans le fichier de journalisation
    with open(log_file, 'a') as f:
        f.write(f"{timestamp} , {message} \n")

                                      Name MC_USD_Billion
0                           JPMorgan Chase         432.92
1                          Bank of America         231.52
2  Industrial and Commercial Bank of China         194.56
3               Agricultural Bank of China         160.68
4                                HDFC Bank         157.91
5                              Wells Fargo         155.87
6                        HSBC Holdings PLC         148.90
7                           Morgan Stanley         140.83
8                  China Construction Bank         139.82
9                            Bank of China         136.81


In [4]:
# fonctions de transformation

def transform(url, data):
    # Création d'une copie du dataframe d'origine pour éviter de le modifier directement.
    df = data.copy()

    # Lecture du fichier CSV contenant les taux de change à partir de l'URL donnée.
    exchange_rate = pd.read_csv(url)

    # Conversion des données du dataframe des taux de change en dictionnaire.
    # La colonne 'Currency' devient les clés du dictionnaire et la colonne 'Rate' devient les valeurs correspondantes.
    exchange_rate = exchange_rate.set_index('Currency').to_dict()['Rate']

    # Conversion des valeurs de 'MC_USD_Billion' en flottants après avoir supprimé les virgules.
    df['MC_USD_Billion'] = df['MC_USD_Billion'].str.replace(',', '').astype(float)

    # Création de nouvelles colonnes pour les valeurs converties en différentes devises.
    # Conversion en GBP (livres sterling) en utilisant le taux de change et en arrondissant à 2 décimales.
    df['MC_GBP_Billion'] = [np.round(x * exchange_rate['GBP'], 2) for x in df['MC_USD_Billion']]

    # Conversion en EUR (euros) en utilisant le taux de change et en arrondissant à 2 décimales.
    df['MC_EUR_Billion'] = [np.round(x * exchange_rate['EUR'], 2) for x in df['MC_USD_Billion']]

    # Conversion en INR (roupies indiennes) en utilisant le taux de change et en arrondissant à 2 décimales.
    df['MC_INR_Billion'] = [np.round(x * exchange_rate['INR'], 2) for x in df['MC_USD_Billion']]

    # Retour du dataframe transformé avec les nouvelles colonnes ajoutées.
    return df

transformed_data = transform(url3, extracted_data)
print(transformed_data)


                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   
5                              Wells Fargo          155.87          124.70   
6                        HSBC Holdings PLC          148.90          119.12   
7                           Morgan Stanley          140.83          112.66   
8                  China Construction Bank          139.82          111.86   
9                            Bank of China          136.81          109.45   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71  
1          215.31        19204.58  
2          180.94        16138.75

In [5]:
# chargement des donnees transformees dans un fichier csv

def load_to_csv( transformed_data, csv_path):
    transformed_data.to_csv(csv_path)

load_to_csv( transformed_data, csv_path) # appel de la fonction


In [6]:
# chargement des donnees du fichier csv dans la base de donnees "Banks.db"
def load_to_db(df, sql_connection, table_name):
    df.to_sql(table_name, sql_connection, if_exists='append', index=False)
    # Enregistre le DataFrame dans une table SQL dela base de donnees , en ajoutant les donnees si la table existe déjà

# appel de la fonction
load_to_db(transformed_data, sql_connection, table_name)

In [7]:
# requettes sur la base de donnees
def run_queries(query_statement, sql_connection):
    # requete formulee
    print(query_statement)

   # Exécute la requête sur la base de données et stocke le résultat dans 'query_statement_output'

    query_statement_output = pd.read_sql(query_statement, sql_connection)

    # afficher le resultat de la requete
    print(query_statement_output)

# formulation de la requete
query_statement= "SELECT Name from Largest_banks LIMIT 5"

# appel de la fonction
run_queries(query_statement, sql_connection)

SELECT Name from Largest_banks LIMIT 5
                                      Name
0                           JPMorgan Chase
1                          Bank of America
2  Industrial and Commercial Bank of China
3               Agricultural Bank of China
4                                HDFC Bank


In [8]:
# Journalisation des activites

log_progress("Debut du processus ETL")

log_progress("Debut d'extraction des donnees")
extracted_data = extract(url, table_attrib)
#print(extracted_data)
log_progress("Fin d'extraction des donnees")

log_progress("Debut de transformation des donnees")
transformed_data = transform(url3, extracted_data)
#print(transformed_data)
log_progress("Fin de transformation des donnees")


log_progress("Debut de chargement des donnees dans un fichier csv")
load_to_csv( transformed_data, csv_path) # appel de la fonction
log_progress("Fin de chargement des donnees dans un fichier csv")


log_progress("Debut de chargement des donnees dans une base donnees")
load_to_db(transformed_data, sql_connection, table_name)
log_progress("Fin de chargement des donnees dans une base donnees")


log_progress("Debut de requetes sur la base donnees")
# formulation de la requete
query_statement= "SELECT Name from Largest_banks LIMIT 5"
# appel de la fonction
run_queries(query_statement, sql_connection)
log_progress("Fin de requetes sur la base donnees")

#

SELECT Name from Largest_banks LIMIT 5
                                      Name
0                           JPMorgan Chase
1                          Bank of America
2  Industrial and Commercial Bank of China
3               Agricultural Bank of China
4                                HDFC Bank


In [9]:
# formulation de la requete
query_statement= "SELECT * FROM Largest_banks"
# appel de la fonction
run_queries(query_statement, sql_connection)

SELECT * FROM Largest_banks
                                       Name  MC_USD_Billion  MC_GBP_Billion  \
0                            JPMorgan Chase          432.92          346.34   
1                           Bank of America          231.52          185.22   
2   Industrial and Commercial Bank of China          194.56          155.65   
3                Agricultural Bank of China          160.68          128.54   
4                                 HDFC Bank          157.91          126.33   
5                               Wells Fargo          155.87          124.70   
6                         HSBC Holdings PLC          148.90          119.12   
7                            Morgan Stanley          140.83          112.66   
8                   China Construction Bank          139.82          111.86   
9                             Bank of China          136.81          109.45   
10                           JPMorgan Chase          432.92          346.34   
11                      

In [10]:
# formulation de la requete
query_statement= "SELECT AVG(MC_GBP_Billion) FROM Largest_banks"
# appel de la fonction
run_queries(query_statement, sql_connection)

SELECT AVG(MC_GBP_Billion) FROM Largest_banks
   AVG(MC_GBP_Billion)
0              151.987


In [11]:
# formulation de la requete
query_statement= "SELECT Name from Largest_banks LIMIT 5"
# appel de la fonction
run_queries(query_statement, sql_connection)

SELECT Name from Largest_banks LIMIT 5
                                      Name
0                           JPMorgan Chase
1                          Bank of America
2  Industrial and Commercial Bank of China
3               Agricultural Bank of China
4                                HDFC Bank


In [12]:
sql_connection.close() # fermeture de la connexion a la base de donnees